# Data Exploration
## Kaggle MS Malware Prediction competition

https://www.kaggle.com/c/microsoft-malware-prediction

## What data do we have?

In [6]:
# Standard useful imports
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

import warnings
import gc
import time
import sys
import datetime
from tqdm import tqdm

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn import metrics

import category_encoders as ce

warnings.simplefilter(action='ignore', category=FutureWarning)

In [7]:
# Create a list that maps list index to column index, with a tuple of column name and description
column_definition = []
# add all the data values from the kaggle dataset
column_definition.append( ("MachineIdentifier" , "Individual machine ID") )
column_definition.append( ("ProductName" , "Defender state information e.g. win8defender") )
column_definition.append( ("EngineVersion" , "Defender state information e.g. 1.1.12603.0") )
column_definition.append( ("AppVersion" , "Defender state information e.g. 4.9.10586.0") )
column_definition.append( ("AvSigVersion" , "Defender state information e.g. 1.217.1014.0") )
column_definition.append( ("IsBeta" , "Defender state information e.g. false") )
column_definition.append( ("RtpStateBitfield" , "NA") )
column_definition.append( ("IsSxsPassiveMode" , "NA") )
column_definition.append( ("DefaultBrowsersIdentifier" , "ID for the machine's default browser") )
column_definition.append( ("AVProductStatesIdentifier" , "ID for the specific configuration of a user's antivirus software") )
column_definition.append( ("AVProductsInstalled" , "NA") )
column_definition.append( ("AVProductsEnabled" , "NA") )
column_definition.append( ("HasTpm" , "True if machine has tpm") )
column_definition.append( ("CountryIdentifier" , "ID for the country the machine is located in") )
column_definition.append( ("CityIdentifier" , "ID for the city the machine is located in") )
column_definition.append( ("OrganizationIdentifier" , "ID for the organization the machine belongs in, organization ID is mapped to both specific companies and broad industries") )
column_definition.append( ("GeoNameIdentifier" , "ID for the geographic region a machine is located in") )
column_definition.append( ("LocaleEnglishNameIdentifier" , "English name of Locale ID of the current user") )
column_definition.append( ("Platform" , "Calculates platform name (of OS related properties and processor property)") )
column_definition.append( ("Processor" , "This is the process architecture of the installed operating system") )
column_definition.append( ("OsVer" , "Version of the current operating system") )
column_definition.append( ("OsBuild" , "Build of the current operating system") )
column_definition.append( ("OsSuite" , "Product suite mask for the current operating system.") )
column_definition.append( ("OsPlatformSubRelease" , "Returns the OS Platform sub-release (Windows Vista, Windows 7, Windows 8, TH1, TH2)") )
column_definition.append( ("OsBuildLab" , "Build lab that generated the current OS. Example: 9600.17630.amd64fre.winblue_r7.150109-2022") )
column_definition.append( ("SkuEdition" , "The goal of this feature is to use the Product Type defined in the MSDN to map to a 'SKU-Edition' name that is useful in population reporting. The valid Product Type are defined in \%sdxroot%\data\windowseditions.xml. This API has been used since Vista and Server 2008, so there are many Product Types that do not apply to Windows 10. The 'SKU-Edition' is a string value that is in one of three classes of results. The design must hand each class.") )
column_definition.append( ("IsProtected" , "This is a calculated field derived from the Spynet Report's AV Products field. Returns: a. TRUE if there is at least one active and up-to-date antivirus product running on this machine. b. FALSE if there is no active AV product on this machine, or if the AV is active, but is not receiving the latest updates. c. null if there are no Anti Virus Products in the report. Returns: Whether a machine is protected.") )
column_definition.append( ("AutoSampleOptIn" , "This is the SubmitSamplesConsent value passed in from the service, available on CAMP 9+") )
column_definition.append( ("PuaMode" , "Pua Enabled mode from the service") )
column_definition.append( ("SMode" , "This field is set to true when the device is known to be in 'S Mode', as in, Windows 10 S mode, where only Microsoft Store apps can be installed") )
column_definition.append( ("IeVerIdentifier" , "NA") )
column_definition.append( ("SmartScreen" , "This is the SmartScreen enabled string value from registry. This is obtained by checking in order, HKLM\\SOFTWARE\\Policies\\Microsoft\\Windows\\System\\SmartScreenEnabled and HKLM\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\Explorer\\SmartScreenEnabled. If the value exists but is blank, the value \"ExistsNotSet\" is sent in telemetry.") )
column_definition.append( ("Firewall" , "This attribute is true (1) for Windows 8.1 and above if windows firewall is enabled, as reported by the service.") )
column_definition.append( ("UacLuaenable" , "This attribute reports whether or not the \"administrator in Admin Approval Mode\" user type is disabled or enabled in UAC. The value reported is obtained by reading the regkey HKLM\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\Policies\\System\\EnableLUA.") )
column_definition.append( ("Census_MDC2FormFactor" , "A grouping based on a combination of Device Census level hardware characteristics. The logic used to define Form Factor is rooted in business and industry standards and aligns with how people think about their device. (Examples: Smartphone, Small Tablet, All in One, Convertible...)") )
column_definition.append( ("Census_DeviceFamily" , "AKA DeviceClass. Indicates the type of device that an edition of the OS is intended for. Example values: Windows.Desktop, Windows.Mobile, and iOS.Phone") )
column_definition.append( ("Census_OEMNameIdentifier" , "NA") )
column_definition.append( ("Census_OEMModelIdentifier" , "NA") )
column_definition.append( ("Census_ProcessorCoreCount" , "Number of logical cores in the processor") )
column_definition.append( ("Census_ProcessorManufacturerIdentifier" , "NA") )
column_definition.append( ("Census_ProcessorModelIdentifier" , "NA") )
column_definition.append( ("Census_ProcessorClass" , "A classification of processors into high/medium/low. Initially used for Pricing Level SKU. No longer maintained and updated") )
column_definition.append( ("Census_PrimaryDiskTotalCapacity" , "Amount of disk space on primary disk of the machine in MB") )
column_definition.append( ("Census_PrimaryDiskTypeName" , "Friendly name of Primary Disk Type" , "HDD or SSD") )
column_definition.append( ("Census_SystemVolumeTotalCapacity" , "The size of the partition that the System volume is installed on in MB") )
column_definition.append( ("Census_HasOpticalDiskDrive" , "True indicates that the machine has an optical disk drive (CD/DVD)") )
column_definition.append( ("Census_TotalPhysicalRAM" , "Retrieves the physical RAM in MB") )
column_definition.append( ("Census_ChassisTypeName" , "Retrieves a numeric representation of what type of chassis the machine has. A value of 0 means xx") )
column_definition.append( ("Census_InternalPrimaryDiagonalDisplaySizeInInches" , "Retrieves the physical diagonal length in inches of the primary display") )
column_definition.append( ("Census_InternalPrimaryDisplayResolutionHorizontal" , "Retrieves the number of pixels in the horizontal direction of the internal display.") )
column_definition.append( ("Census_InternalPrimaryDisplayResolutionVertical" , "Retrieves the number of pixels in the vertical direction of the internal display") )
column_definition.append( ("Census_PowerPlatformRoleName" , "Indicates the OEM preferred power management profile. This value helps identify the basic form factor of the device") )
column_definition.append( ("Census_InternalBatteryType" , "NA") )
column_definition.append( ("Census_InternalBatteryNumberOfCharges" , "NA") )
column_definition.append( ("Census_OSVersion" , "Numeric OS version Example" , "10.0.10130.0") )
column_definition.append( ("Census_OSArchitecture" , "Architecture on which the OS is based. Derived from OSVersionFull. Example" , "amd64") )
column_definition.append( ("Census_OSBranch" , "Branch of the OS extracted from the OsVersionFull. Example" , "OsBranch = fbl_partner_eeap where OsVersion = 6.4.9813.0.amd64fre.fbl_partner_eeap.140810-0005") )
column_definition.append( ("Census_OSBuildNumber" , "OS Build number extracted from the OsVersionFull. Example" , "OsBuildNumber = 10512 or 10240") )
column_definition.append( ("Census_OSBuildRevision" , "OS Build revision extracted from the OsVersionFull. Example" , "OsBuildRevision = 1000 or 16458") )
column_definition.append( ("Census_OSEdition" , "Edition of the current OS. Sourced from HKLM\Software\Microsoft\Windows NT\CurrentVersion@EditionID in registry. Example: Enterprise") )
column_definition.append( ("Census_OSSkuName" , "OS edition friendly name (currently Windows only)") )
column_definition.append( ("Census_OSInstallTypeName" , "Friendly description of what install was used on the machine i.e. clean") )
column_definition.append( ("Census_OSInstallLanguageIdentifier" , "NA") )
column_definition.append( ("Census_OSUILocaleIdentifier" , "NA") )
column_definition.append( ("Census_OSWUAutoUpdateOptionsName" , "Friendly name of the WindowsUpdate auto-update settings on the machine.") )
column_definition.append( ("Census_IsPortableOperatingSystem" , "Indicates whether OS is booted up and running via Windows-To-Go on a USB stick.") )
column_definition.append( ("Census_GenuineStateName" , "Friendly name of OSGenuineStateID. 0 = Genuine") )
column_definition.append( ("Census_ActivationChannel" , "Retail license key or Volume license key for a machine.") )
column_definition.append( ("Census_IsFlightingInternal" , "NA") )
column_definition.append( ("Census_IsFlightsDisabled" , "Indicates if the machine is participating in flighting.") )
column_definition.append( ("Census_FlightRing" , "The ring that the device user would like to receive flights for. This might be different from the ring of the OS which is currently installed if the user changes the ring after getting a flight from a different ring.") )
column_definition.append( ("Census_ThresholdOptIn" , "NA") )
column_definition.append( ("Census_FirmwareManufacturerIdentifier" , "NA") )
column_definition.append( ("Census_FirmwareVersionIdentifier" , "NA") )
column_definition.append( ("Census_IsSecureBootEnabled" , "Indicates if Secure Boot mode is enabled.") )
column_definition.append( ("Census_IsWIMBootEnabled" , "NA") )
column_definition.append( ("Census_IsVirtualDevice" , "Identifies a Virtual Machine (machine learning model)") )
column_definition.append( ("Census_IsTouchEnabled" , "Is this a touch device ?") )
column_definition.append( ("Census_IsPenCapable" , "Is the device capable of pen input ?") )
column_definition.append( ("Census_IsAlwaysOnAlwaysConnectedCapable" , "Retreives information about whether the battery enables the device to be AlwaysOnAlwaysConnected .") )
column_definition.append( ("Wdft_IsGamer" , "Indicates whether the device is a gamer device or not based on its hardware combination.") )
column_definition.append( ("Wdft_RegionIdentifier" , "NA") )

# We can now create a data types dictionary to support loading data in a more efficient format
data_types = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

# https://www.kaggle.com/theoviel/load-the-totality-of-the-data
# https://www.kaggle.com/airbourne/data-dictionary

## Load data

In this challenge, we are given two files:
* train.csv
* test.csv

test.csv does not include lables - this is used to create a submission for evaluation by kaggle, for the leaderboard.


In [19]:
num_rows = 100000

train = pd.read_csv('/Users/amb/dev/kaggle/malware_prediction/data/train.csv',
#                     nrows = num_rows,
                    dtype = data_types)

# separate out labels from data
labels = train.HasDetections.copy()
train = train.drop(columns = ["HasDetections"])

print("Size of training dataset dataframe = {}".format(train.size))

print("Size of label dataframe = {}".format(labels.size))

NameError: name 'train_test_split' is not defined

In [9]:
print(train.dtypes)

MachineIdentifier                          category
ProductName                                category
EngineVersion                              category
AppVersion                                 category
AvSigVersion                               category
IsBeta                                         int8
RtpStateBitfield                            float16
IsSxsPassiveMode                               int8
DefaultBrowsersIdentifier                   float16
AVProductStatesIdentifier                   float32
AVProductsInstalled                         float16
AVProductsEnabled                           float16
HasTpm                                         int8
CountryIdentifier                             int16
CityIdentifier                              float32
OrganizationIdentifier                      float16
GeoNameIdentifier                           float16
LocaleEnglishNameIdentifier                    int8
Platform                                   category
Processor   

In [10]:
train.head(4)

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,...,628.0,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,53447.0,...,628.0,57858.0,0,NaN,0.0,0,0,0.0,0.0,8.0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,0,7.0,0,NaN,53447.0,...,142.0,52682.0,0,NaN,0.0,0,0,0.0,0.0,3.0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,...,355.0,20050.0,0,NaN,0.0,0,0,0.0,0.0,3.0


In [11]:
missing_data_counts = train.isna().sum().to_dict()

sorted_missing_data = [ (k, missing_data_counts[k]) for k in sorted(missing_data_counts, key=missing_data_counts.get, reverse=True) ]

print("Top 10 variable for missing data")
for idx in range(0,10):
    print("\t{}".format(sorted_missing_data[idx]))

Top 10 variable for missing data
	('PuaMode', 199947)
	('Census_ProcessorClass', 199127)
	('DefaultBrowsersIdentifier', 190297)
	('Census_IsFlightingInternal', 166068)
	('Census_InternalBatteryType', 142088)
	('Census_ThresholdOptIn', 127008)
	('Census_IsWIMBootEnabled', 126839)
	('SmartScreen', 71123)
	('OrganizationIdentifier', 61537)
	('SMode', 11863)


In [12]:
# insepct each column to count unique values
unique_counts = {}

for idx in range(0, train.shape[1]):
    col_name = train.columns[idx]
    unique_counts[col_name] = train[col_name].nunique()
#     print("{} unique values in {}".format(unique_counts[col_name] , col_name))

sorted_unique_counts = [ (k , unique_counts[k]) for k in sorted(unique_counts, key=unique_counts.get, reverse=True) ]

print("\nTop 5 cardinaliy variables, sorted (high to low):")
for idx in range(0, 5):
    k,v = sorted_unique_counts[idx]
    print("\t* {} unique values in {}".format(v, k) )

possible_binary_variables = [c for c in train.columns if (train[c].nunique() == 2 or train[c].nunique() == 3) ]

print("\nPossible binary variables:")
for column in possible_binary_variables:
    print("\t* {} values = {}".format(column, train[column].value_counts().to_dict()))




Top 5 cardinaliy variables, sorted (high to low):
	* 200000 unique values in MachineIdentifier
	* 76808 unique values in Census_SystemVolumeTotalCapacity
	* 24416 unique values in Census_OEMModelIdentifier
	* 23378 unique values in CityIdentifier
	* 16797 unique values in Census_FirmwareVersionIdentifier

Possible binary variables:
	* ProductName values = {'win8defender': 197839, 'mse': 2161}
	* IsSxsPassiveMode values = {0: 196582, 1: 3418}
	* HasTpm values = {1: 197536, 0: 2464}
	* Processor values = {'x64': 181693, 'x86': 18294, 'arm64': 13}
	* IsProtected values = {1.0: 188299, 0.0: 10902}
	* AutoSampleOptIn values = {0: 199993, 1: 7}
	* SMode values = {0.0: 188072, 1.0: 65}
	* Firewall values = {1.0: 193528, 0.0: 4394}
	* Census_DeviceFamily values = {'Windows.Desktop': 199682, 'Windows.Server': 318}
	* Census_ProcessorClass values = {'mid': 490, 'low': 256, 'high': 127}
	* Census_HasOpticalDiskDrive values = {0: 184705, 1: 15295}
	* Census_OSArchitecture values = {'amd64': 18170

# Assessing our Training Dataset

The training dataset contains 82 variables, and most of them are categorical variables.

To make progress here, we need to perform feature encoding - converting categorical variables to a meaningful numerical represenation to enable ML models and algorithms to function.


## Categorical variables & feature encoding

Converting categorical variables into a numerical representation suitable for ML is a challenge.

There are many possible methods for such converstions, and different conversion schemes are suitable for different kinds of data and application.  We look at four techniques, split across low and high cardinality scenarios.


### Label Encoding & One-Hot encoding (LOW cardinality)

Two very common, widely used encoding methods are label encoding and one-hot encoding.

Label encoding can introduce artificial strucutre into a dataset, where categorical values are treated as rational numerical values, with a meaningful definition of order and distance between values.

One-hot methods are best suited to variables with low cardility (i.e. low number of unique values) because the technique maps and thereby increases the dimensionality of the dataset.


### Frequency Encoding & Hash Encoding (HIGH cardinality)

In the case of categorical variables that have high cardinality - that is, many unique possible values - techniques that convert from values to variables will massively increase the dimensionality of the dataset.

The curse of dimensionality is the term that refers to the computational and representational challenges that are inherent in high dimensional datsets. We should seek to reduce dimensionality where practical, and avoid increasing dataset dimensionality.

We see that label encoding is still viable, replacing values with integers.  This keeps dimensionality and cardiality the same.

One-hot encoding increases the dimensionality of the dataset by the cardinality of the variable - something that is very undesirable when dealing with high-cardinality varaibles.

Two widely used methods for dealing with high-cardinality variables are are: frequency encoding and hash encoding.

Frequency encoding replaces a categorical value with the frequency (count) of the value within the training dataset.

Hash encoding uses a hash function to reduce high cardinality function to a predefined smaller vector.



### Ordinal Variables

In the case of ordinal data (categorical data with meaningful definition of rank, or order) a simple technique to replace with ordered numbers e.g { high, med, low } => {9, 3, 1} 

Such data driven approaches fail when a new value is seen in a prediction i.e. a value that has not appeared in the training dataset.

### Unseen Category Values

Where possible, we should seek to define the full set of all valid category values up front.  If this is not possible, it is possible to create an "unknown" value in label and on-feature-hot encoding to represent unseen variables. 


## Other Obseravations

### Binary variables

The dataset also contains several binary variables, with values { 0 , 1 }. These are:
* IsBeta value counts = {0: 8921416, 1: 67}
* IsSxsPassiveMode value counts = {0: 8766840, 1: 154643}
* HasTpm value counts = {1: 8814167, 0: 107316}
* Processor value counts = {'x64': 8105435, 'x86': 815702, 'arm64': 346}
* IsProtected value counts = {1.0: 8402282, 0.0: 483157}
* AutoSampleOptIn value counts = {0: 8921225, 1: 258}
* PuaMode value counts = {'on': 2307, 'audit': 2}
* SMode value counts = {0.0: 8379843, 1.0: 3881}
* Firewall value counts = {1.0: 8641014, 0.0: 189119}
* Census_HasOpticalDiskDrive value counts = {0: 8232858, 1: 688625}
* Census_IsPortableOperatingSystem value counts = {0: 8916619, 1: 4864}
* Census_IsFlightingInternal value counts = {0.0: 1512703, 1.0: 21}
* Census_IsFlightsDisabled value counts = {0.0: 8760872, 1.0: 88}
* Census_ThresholdOptIn value counts = {0.0: 3253342, 1.0: 816}
* Census_IsSecureBootEnabled value counts = {0: 4585438, 1: 4336045}
* Census_IsWIMBootEnabled value counts = {0.0: 3261779, 1.0: 1}
* Census_IsVirtualDevice value counts = {0.0: 8842840, 1.0: 62690}
* Census_IsTouchEnabled value counts = {0: 7801452, 1: 1120031}
* Census_IsPenCapable value counts = {0: 8581834, 1: 339649}
* Census_IsAlwaysOnAlwaysConnectedCapable value counts = {0.0: 8341972, 1.0: 508168}
* Wdft_IsGamer value counts = {0.0: 6174143, 1.0: 2443889}
* HasDetections value counts = {0: 4462591, 1: 4458892}


### Missing data

The dataset contains some missing data.
Top 10 variables by number of missing data values:
* ('PuaMode', 8919174)
* ('Census_ProcessorClass', 8884852)
* ('DefaultBrowsersIdentifier', 8488045)
* ('Census_IsFlightingInternal', 7408759)
* ('Census_InternalBatteryType', 6338429)
* ('Census_ThresholdOptIn', 5667325)
* ('Census_IsWIMBootEnabled', 5659703)
* ('SmartScreen', 3177011)
* ('OrganizationIdentifier', 2751518)
* ('SMode', 537759)



## Can we save memory and compute time by reducing dataset size?

When we loaded the CSV file using pandas, the library makes sensible mapping between file content and data type.

These mapping can often be imroved in order to use less memory (e.g. low numbers converted from float64 to uint8).

Some of the values may be unnecessary/unhelpful and therefore can be deleted.  Other variables may have long categorical values that can be replaced with small integers.


Here we review the training dataset to see where it is possible to define categorical variables, use small integer data types, remove unnecessary/duplicative variables and replace large values with smaller values.

See here for very similar work:

https://www.kaggle.com/fabiendaniel/detecting-malwares-with-lgbm

# Data Preparation

Here is our workfow:

1. Load data from CSV using minimal data types

1. **Split labels from data**

1. **Remove unnecessary variables**
  1. **MachineIdentifier** - autogenerated identifier, provides no info about machine or threat
  1. **DefaultBrowserIdentifier** - large number of NaN values; potential to add in later
  1. **CityIdentifier** - drop as very high cardiality, and lots of NaNs, retain geo info from GeoNameIdentifier
  1. **CountryIdentifier** - drop as very high cardinality, retain geo info from GeoNameIdentifier
  1. **OrganizationIdenifier** - drop in first iteration
  1. **LocaleEnglishNameIdentifier** - drop in first iteration
  1. **OsSuite** - drop in first iteration, as has very skewed distro, related info in OsPlatformSubRelease
  1. **Census_OEMNameIdentifier** - drop
  1. **Census_ProcessorModelIdentifier** - drop
  1. **Census_ProcessorClass** - drop old variable, also lots of missing values
  1. **Census_InternalPrimaryDiagonalDisplaySizeInInches** - drop
  1. **Census_InternalPrimaryDisplayResolutionHorizontal** - drop
  1. **Census_InternalPrimaryDisplayResolutionVertical** - drop
  1. **Census_PowerPlatformRoleName** - drop
  1. **Census_InternalBatteryType** - drop
  1. **Census_InternalBatteryNumberOfCharges** - drop
  1. **Census_OSSkuName** - drop
  1. **Census_ActivationChannel** - drop
  1. **Census_FirmwareManufacturerIdentifier** - drop, potentially frequency encode/hash encode
  1. **Census_FirmwareVersionIdentifier** - 50k values, drop
  1. **Census_FlightRing** - drop, potentially freq hash encode
  1. **PuaMode** - drop due to missing data (8919174 missing values)
  

1. **One-hot encode** low cardinality (>2, <5) variables
  1. **Platform** - 4 values, potential to use freq encoding later
  1. **Census_DeviceFamily** - 3 unique values

1. **Frequency encode** variables
  1. **GeoNameIdentifier** - 292 values
  1. **SkuEdition** - tells us about the edition of the OS, and likely an important indicator of use, protection & threat
  1. **Census_ProcessorCoreCount** - indicator about machine cost, common to rare
  1. **Census_TotalPhysicalRAM** - indicator about machine cost, common to rare
  1. **Census_ChassisTypeName**
  1. **Wdft_RegionIdentifier** - geographic region
  1. **Census_OSInstallTypeName** - 

1. **Hash encode** similar/related high cardinality (>=5) variables
  1. **DefaultBrowsersIdentifier** to 5 features (major browser categories?)
  1. OS description variables - let's reduce this to 6 features to describe OS
      1. Census_OSVersion
      1. Census_OSArchitecture
      1. Census_OSBranch
      1. Census_OSBuildNumber
      1. Census_OSBuildRevision
      1. Census_OSEdition
      1. Census_OSInstallTypeName
      1. Census_OSInstallLanguageIdentifier
      1. Census_OSUILocaleIdentifier
      1. Census_OSWUAutoUpdateOptionsName
  1. Defender description variables - lets reduce this to 7 features to describe defender state
      1. **ProductName**
      1. **EngineVersion**
      1. **AppVersion**
      1. **AvSigVersion**
  
1. Normalise training dataset (subtact mean, divide by std dev for each column)



In [13]:
all_vars = { 'MachineIdentifier','ProductName','EngineVersion',
        'AppVersion','AvSigVersion','IsBeta','RtpStateBitfield',
        'IsSxsPassiveMode','DefaultBrowsersIdentifier','AVProductStatesIdentifier',
        'AVProductsInstalled','AVProductsEnabled','HasTpm','CountryIdentifier',
        'CityIdentifier','OrganizationIdentifier','GeoNameIdentifier','LocaleEnglishNameIdentifier',
        'Platform','Processor','OsVer','OsBuild','OsSuite','OsPlatformSubRelease','OsBuildLab',
        'SkuEdition','IsProtected','AutoSampleOptIn','PuaMode','SMode','IeVerIdentifier',
        'SmartScreen','Firewall','UacLuaenable','Census_MDC2FormFactor','Census_DeviceFamily',
        'Census_OEMNameIdentifier','Census_OEMModelIdentifier','Census_ProcessorCoreCount',
        'Census_ProcessorManufacturerIdentifier','Census_ProcessorModelIdentifier',
        'Census_ProcessorClass','Census_PrimaryDiskTotalCapacity','Census_PrimaryDiskTypeName',
        'Census_SystemVolumeTotalCapacity','Census_HasOpticalDiskDrive','Census_TotalPhysicalRAM',
        'Census_ChassisTypeName','Census_InternalPrimaryDiagonalDisplaySizeInInches',
        'Census_InternalPrimaryDisplayResolutionHorizontal','Census_InternalPrimaryDisplayResolutionVertical',
        'Census_PowerPlatformRoleName','Census_InternalBatteryType','Census_InternalBatteryNumberOfCharges',
        'Census_OSVersion','Census_OSArchitecture','Census_OSBranch','Census_OSBuildNumber',
        'Census_OSBuildRevision','Census_OSEdition','Census_OSSkuName','Census_OSInstallTypeName',
        'Census_OSInstallLanguageIdentifier','Census_OSUILocaleIdentifier','Census_OSWUAutoUpdateOptionsName',
        'Census_IsPortableOperatingSystem','Census_GenuineStateName','Census_ActivationChannel',
        'Census_IsFlightingInternal','Census_IsFlightsDisabled','Census_FlightRing',
        'Census_ThresholdOptIn','Census_FirmwareManufacturerIdentifier',
        'Census_FirmwareVersionIdentifier','Census_IsSecureBootEnabled','Census_IsWIMBootEnabled',
        'Census_IsVirtualDevice','Census_IsTouchEnabled','Census_IsPenCapable',
        'Census_IsAlwaysOnAlwaysConnectedCapable','Wdft_IsGamer','Wdft_RegionIdentifier'
    }

one_hot_encode_vars = {
                        "Census_DeviceFamily"
                    }

freq_encode_vars = {
                    "SkuEdition","GeoNameIdentifier",
                    "Wdft_RegionIdentifier",
                    "Firewall","Platform",
                    "Census_ProcessorCoreCount","Census_TotalPhysicalRAM",
                    "Census_ChassisTypeName"
                    }

hash_encode_os_vars = {
                        "Census_OSVersion","Census_OSArchitecture",
                        "Census_OSBranch","Census_OSBuildNumber",
                        "Census_OSBuildRevision","Census_OSEdition",
                        "Census_OSInstallTypeName","Census_OSInstallLanguageIdentifier",
                        "Census_OSUILocaleIdentifier","Census_OSWUAutoUpdateOptionsName"
                    }

hash_encode_defender_status_vars = ["ProductName","EngineVersion",
                                    "AppVersion","AvSigVersion"]

processed_var_count = len(one_hot_encode_vars) +len(freq_encode_vars) + len(hash_encode_os_vars) + len(hash_encode_defender_status_vars)

print("Handling {} variables; {} remain".format(
                processed_var_count , (len(all_vars) - processed_var_count ) ) )

Handling 23 variables; 59 remain


In [14]:
def create_frequency_encoding(df, variable_list):
    
    freq_encoding = {}
    
    for column in variable_list:
        freq_encoding[column] = df[column].value_counts().to_dict()
        
    return freq_encoding
    

## Data processing function
* **Delete variables**
* **Frequency encode variables**
* **Hash encode variables**

In [15]:
def process_dataframe(df):
    
    # freq encode variables
    for column in freq_encode_vars:
        value_counts = df[column].value_counts().to_dict()
        df[column] = df[column].map(value_counts)

    remaining_vars = all_vars.difference(freq_encode_vars)
    print("Frequency encoding {} vars completed".format(len(freq_encode_vars)))
    
    # one-hot encode variables
    df = pd.get_dummies(df, columns= list(one_hot_encode_vars) )
    print("One-hot encoding {} vars completed".format(len(one_hot_encode_vars)))
    remaining_vars = remaining_vars.difference(one_hot_encode_vars)
    # hash encode variables
    
    # hash encode OS census vars
    df = ce.HashingEncoder.hashing_trick(df, N=6, cols= list(hash_encode_os_vars) )
    remaining_vars = remaining_vars.difference(hash_encode_os_vars)
    print("OS hash encoding {} vars completed".format(len(hash_encode_os_vars)))
    
    # hash encode defender stat vars
    df = ce.HashingEncoder.hashing_trick(df, N=4, cols= list(hash_encode_defender_status_vars))
    remaining_vars = remaining_vars.difference(hash_encode_defender_status_vars)
    print("Defender hash encoding {} vars completed".format(len(hash_encode_defender_status_vars)))
    
    # remove variables
    print("{} variables left to process".format(len(remaining_vars)))
    df = df.drop(columns = list(remaining_vars) )
    print("{} variables deleted".format(len(remaining_vars)))
    
    return df

In [16]:
def process_new_input(input):
    # convert input into model-compatible form
    return 42

In [17]:
# create frequency encoding map
freq_encoding_map = create_frequency_encoding(train, freq_encode_vars)
print("Frequency encoding map contains {} entries".format(len(freq_encoding_map.keys())))

Frequency encoding map contains 8 entries


# Baseline classifier



In [18]:
from sklearn import svm

from sklearn.impute import SimpleImputer

from sklearn.metrics import accuracy_score


# process dataframe
train2 = process_dataframe(train)


# lets replace all missing values with the most frequent value in the column
imp = SimpleImputer(missing_values=np.NaN, strategy='most_frequent')
imp.fit(train2)
train2 = imp.transform(train2)

clf = svm.SVC(gamma='scale')

print("\nNow training SVM classifier on training dataset...")
clf.fit(train2, labels)
print("Training complete.\nComputing training accuracy score...")
y_pred = clf.predict(train2)
accuracy = accuracy_score(labels, y_pred)

print("Accuracy (train) for SVM: {}%".format(accuracy * 100))
print("Completed.")
    

Frequency encoding 8 vars completed
One-hot encoding 1 vars completed
OS hash encoding 10 vars completed
Defender hash encoding 4 vars completed
59 variables left to process
59 variables deleted

Now training SVM classifier on training dataset...
Training complete.
Computing training accuracy score...
Accuracy (train) for SVM: 60.32149999999999%
Completed.


# Can we reduce data dimensionality?

Now we've reduced the dataset size using smaller data types, removed unnecessary varaibles, performed feature encoding and normalised the training data, let's see if we can apply data reduction techniques to reduce the dataset even more and maintain sufficient information to support learning and prediction.
* PCA
* t-SNE
* Are any of the variables highly correlated?

## PCA and T-SNE

PCA is a technique for reducing the number of dimensions in a dataset whilst retaining most information. It uses the correlation between some dimensions and tries to provide a minimum number of variables that keeps the maximum amount of variation or information about how the original data is distributed.


In [8]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca_result = pca.fit_transform(df['train2'].values)

train2['pca-one'] = pca_result[:,0]
train2['pca-two'] = pca_result[:,1] 
train2['pca-three'] = pca_result[:,2]

print("Explained variation per principal component: {}").format(pca.explained_variance_ratio_)

NameError: name 'df' is not defined